In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation

# importe le simulateur depuis simulateur.py
from simulateur import *

# importe l'algorithme RRT*
from PythonRobotics.PathPlanning.RRTStar.rrt_star import RRTStar

from helpers import *

### Préparer la carte

In [ ]:
# Cell Dimension: 633, 1149

# importing the map and droping the last empty column
df = pd.read_csv('maps/RDCmod3.txt', header=None)
df = df.drop(columns=[633])

# droping columns and rows with 0
df = df.loc[:, (df != 0).any(axis=0)]
df = df.loc[(df!=0).any(axis=1)]

# reverse the order of the DataFrame, to match the reality
df = df.iloc[::-1]
df.reset_index(inplace=True, drop=True)

#normalizing the dataframe
df_norm = -df.copy()
for column in df_norm.columns:
    df_norm[column] = df_norm[column] / df_norm[column].abs().max()
    
#Rounding the dataframe

#tester en gardant le vide dans la carte, soulage le simulateur ?
df_rounded = df_norm.round(0).astype(int)
df_rounded = df_rounded + 1
df_rounded.replace(2,1, inplace=True)

#Exporting the final map
df_rounded.to_csv("maps/mapnorm.csv",sep=';', header=False, index=False)

In [ ]:
dftest = pd.read_csv("maps/mapnorm.csv",sep=';', header=None)

In [ ]:
dftest2 = dftest.copy()

In [ ]:
dftest2

In [ ]:
for y in range(len(dftest)):
    for x in range(len(dftest.iloc[y])):
        if not y==0 and not x==0 and not y==681 and not x==252:
            if(dftest.iloc[y-1][x] == 1 and dftest.iloc[y+1][x] == 1 and dftest.iloc[y][x-1] == 1 and dftest.iloc[y][x+1] == 1):
                dftest2.iloc[y][x] = 0
        else:
            dftest2.iloc[y][x] = 0

In [ ]:
dftest2.to_csv("maps/mapnorm1.csv",sep=';', header=False, index=False)

### Simulation

In [78]:
# room = DogRoom("maps/map01.csv")
room = DogRoom("maps/mapnorm1.csv")
dog = Dog(room, speed=1)

In [ ]:
room.plotmap()

In [ ]:
# Crée les obstacles
obstacle = generateObstacle(room, 1.2)

In [79]:
showmap(room = room, dog = dog)

In [ ]:
%matplotlib

#définit la destination
goal = [12,5]
# goal = [25, 375]

#crée le chemin
rrt_star = RRTStar(
        start=[dog.x, dog.y],
        goal=goal,
        rand_area=[2, 15],
        obstacle_list = obstacle,
        goal_sample_rate=1,
        expand_dis=5,
        max_iter=200,
        search_until_max_iter=True)
path = rrt_star.planning(animation=False)
if path is None:
    print("Cannot find path")
else:
    print("found path!!")
# path = generatepath(room, dog, goal, obstacle_list, max_iter=1000, search_until_max_iter=False)
showmap(room, dog = dog, goal = goal,  obstacle = obstacle, path=path)
#parcourt le chemin
# MoveUsingPath(dog, path)

In [54]:
path

[[12, 5],
 [7.625359146241661, 5.961212906667916],
 [4.23675724676926, 8.84985011677832],
 [5.611792232518766, 12.828471985236966],
 [7.081114786528113, 15.940429935254823],
 [10, 20]]

In [ ]:
MoveUsingPath(room, dog, path)

In [ ]:
dog.rotate(90)

In [ ]:
dog.forward(-2)

In [65]:
"""
Env 2D
@author: huiming zhou
"""


class Env:
    def __init__(self):
        self.x_range = room.largeur  # size of background
        self.y_range = room.longueur
        self.motions = [(-1, 0), (-1, 1), (0, 1), (1, 1),
                        (1, 0), (1, -1), (0, -1), (-1, -1)]
        self.obs = self.obs_map()

    def update_obs(self, obs):
        self.obs = obs

    def obs_map(self):
        """
        Initialize obstacles' positions
        :return: map of obstacles
        """

        x = self.x_range
        y = self.y_range
        obs = set()

        for y in range(len(room.imgdf)):
            for x in range(len(room.imgdf.iloc[y])):
                if room.imgdf.iloc[y][x]==1:
                    obs.add((x,y))
                    obs.add((x-1,y))
                    obs.add((x+1,y))
                    obs.add((x,y+1))
                    obs.add((x,y-1))
        return obs


In [66]:

class Plotting:
    def __init__(self, xI, xG):
        self.xI, self.xG = xI, xG
        self.env = Env()
        self.obs = self.env.obs_map()

    def update_obs(self, obs):
        self.obs = obs

    def animation(self, path, visited, name):
        self.plot_grid(name)
        self.plot_visited(visited)
        self.plot_path(path)
        plt.show()

    def animation_lrta(self, path, visited, name):
        self.plot_grid(name)
        cl = self.color_list_2()
        path_combine = []

        for k in range(len(path)):
            self.plot_visited(visited[k], cl[k])
            plt.pause(0.2)
            self.plot_path(path[k])
            path_combine += path[k]
            plt.pause(0.2)
        if self.xI in path_combine:
            path_combine.remove(self.xI)
        self.plot_path(path_combine)
        plt.show()

    def animation_ara_star(self, path, visited, name):
        self.plot_grid(name)
        cl_v, cl_p = self.color_list()

        for k in range(len(path)):
            self.plot_visited(visited[k], cl_v[k])
            self.plot_path(path[k], cl_p[k], True)
            plt.pause(0.5)

        plt.show()

    def animation_bi_astar(self, path, v_fore, v_back, name):
        self.plot_grid(name)
        self.plot_visited_bi(v_fore, v_back)
        self.plot_path(path)
        plt.show()

    def plot_grid(self, name):
        obs_x = [x[0] for x in self.obs]
        obs_y = [x[1] for x in self.obs]

        plt.plot(self.xI[0], self.xI[1], "bs")
        plt.plot(self.xG[0], self.xG[1], "gs")
        plt.plot(obs_x, obs_y, "sk")
        plt.title(name)
        plt.axis("equal")

    def plot_visited(self, visited, cl='gray'):
        if self.xI in visited:
            visited.remove(self.xI)

        if self.xG in visited:
            visited.remove(self.xG)

        count = 0

        for x in visited:
            count += 1
            plt.plot(x[0], x[1], color=cl, marker='o')
            plt.gcf().canvas.mpl_connect('key_release_event',
                                         lambda event: [exit(0) if event.key == 'escape' else None])

            if count < len(visited) / 3:
                length = 20
            elif count < len(visited) * 2 / 3:
                length = 30
            else:
                length = 40
            #
            # length = 15

            if count % length == 0:
                plt.pause(0.001)
        plt.pause(0.01)

    def plot_path(self, path, cl='r', flag=False):
        path_x = [path[i][0] for i in range(len(path))]
        path_y = [path[i][1] for i in range(len(path))]

        if not flag:
            plt.plot(path_x, path_y, linewidth='3', color='r')
        else:
            plt.plot(path_x, path_y, linewidth='3', color=cl)

        plt.plot(self.xI[0], self.xI[1], "bs")
        plt.plot(self.xG[0], self.xG[1], "gs")

        plt.pause(0.01)

    def plot_visited_bi(self, v_fore, v_back):
        if self.xI in v_fore:
            v_fore.remove(self.xI)

        if self.xG in v_back:
            v_back.remove(self.xG)

        len_fore, len_back = len(v_fore), len(v_back)

        for k in range(max(len_fore, len_back)):
            if k < len_fore:
                plt.plot(v_fore[k][0], v_fore[k][1], linewidth='3', color='gray', marker='o')
            if k < len_back:
                plt.plot(v_back[k][0], v_back[k][1], linewidth='3', color='cornflowerblue', marker='o')

            plt.gcf().canvas.mpl_connect('key_release_event',
                                         lambda event: [exit(0) if event.key == 'escape' else None])

            if k % 10 == 0:
                plt.pause(0.001)
        plt.pause(0.01)

    @staticmethod
    def color_list():
        cl_v = ['silver',
                'wheat',
                'lightskyblue',
                'royalblue',
                'slategray']
        cl_p = ['gray',
                'orange',
                'deepskyblue',
                'red',
                'm']
        return cl_v, cl_p

    @staticmethod
    def color_list_2():
        cl = ['silver',
              'steelblue',
              'dimgray',
              'cornflowerblue',
              'dodgerblue',
              'royalblue',
              'plum',
              'mediumslateblue',
              'mediumpurple',
              'blueviolet',
              ]
        return cl

In [ ]:
env = Env()

In [90]:
"""
D_star_Lite 2D
@author: huiming zhou
"""

import os
import sys
import math
import matplotlib.pyplot as plt

# sys.path.append(os.path.dirname(os.path.abspath(__file__)) +
#                 "/../../Search_based_Planning/")

# from PathPlanning.Search_based_Planning.Search_2D import plotting


class DStar:
    def __init__(self, s_start, s_goal, heuristic_type):
        self.s_start, self.s_goal = s_start, s_goal
        self.heuristic_type = heuristic_type

        self.Env = Env()  # class Env
        self.Plot = Plotting(s_start, s_goal)

        self.u_set = self.Env.motions  # feasible input set
        self.obs = self.Env.obs  # position of obstacles
        self.x = self.Env.x_range
        self.y = self.Env.y_range

        self.g, self.rhs, self.U = {}, {}, {}
        self.km = 0

        for i in range(1, self.Env.x_range - 1):
            for j in range(1, self.Env.y_range - 1):
                self.rhs[(i, j)] = float("inf")
                self.g[(i, j)] = float("inf")

        self.rhs[self.s_goal] = 0.0
        self.U[self.s_goal] = self.CalculateKey(self.s_goal)
        self.visited = set()
        self.count = 0
        self.fig = plt.figure()

    def run(self):
        self.Plot.plot_grid("D* Lite")
        self.ComputePath()
        self.plot_path(self.extract_path())
        self.fig.canvas.mpl_connect('button_press_event', self.on_press)
        plt.show()

    def on_press(self, event):
        x, y = event.xdata, event.ydata
        if x < 0 or x > self.x - 1 or y < 0 or y > self.y - 1:
            print("Please choose right area!")
        else:
            x, y = int(x), int(y)
            print("Change position: s =", x, ",", "y =", y)

            s_curr = self.s_start
            s_last = self.s_start
            i = 0
            path = [self.s_start]

            while s_curr != self.s_goal:
                s_list = {}

                for s in self.get_neighbor(s_curr):
                    s_list[s] = self.g[s] + self.cost(s_curr, s)
                s_curr = min(s_list, key=s_list.get)
                path.append(s_curr)

                if i < 1:
                    self.km += self.h(s_last, s_curr)
                    s_last = s_curr
                    if (x, y) not in self.obs:
                        self.obs.add((x, y))
                        plt.plot(x, y, 'sk')
                        self.g[(x, y)] = float("inf")
                        self.rhs[(x, y)] = float("inf")
                    else:
                        self.obs.remove((x, y))
                        plt.plot(x, y, marker='s', color='white')
                        self.UpdateVertex((x, y))
                    for s in self.get_neighbor((x, y)):
                        self.UpdateVertex(s)
                    i += 1

                    self.count += 1
                    self.visited = set()
                    self.ComputePath()

            # self.plot_visited(self.visited)
            self.plot_path(path)
            self.fig.canvas.draw_idle()

    def ComputePath(self):
        while True:
            s, v = self.TopKey()
            if v >= self.CalculateKey(self.s_start) and \
                    self.rhs[self.s_start] == self.g[self.s_start]:
                break

            k_old = v
            self.U.pop(s)
            self.visited.add(s)

            if k_old < self.CalculateKey(s):
                self.U[s] = self.CalculateKey(s)
            elif self.g[s] > self.rhs[s]:
                self.g[s] = self.rhs[s]
                for x in self.get_neighbor(s):
                    self.UpdateVertex(x)
            else:
                self.g[s] = float("inf")
                self.UpdateVertex(s)
                for x in self.get_neighbor(s):
                    self.UpdateVertex(x)

    def UpdateVertex(self, s):
        if s != self.s_goal:
            self.rhs[s] = float("inf")
            for x in self.get_neighbor(s):
                self.rhs[s] = min(self.rhs[s], self.g[x] + self.cost(s, x))
        if s in self.U:
            self.U.pop(s)

        if self.g[s] != self.rhs[s]:
            self.U[s] = self.CalculateKey(s)

    def CalculateKey(self, s):
        return [min(self.g[s], self.rhs[s]) + self.h(self.s_start, s) + self.km,
                min(self.g[s], self.rhs[s])]

    def TopKey(self):
        """
        :return: return the min key and its value.
        """

        s = min(self.U, key=self.U.get)
        return s, self.U[s]

    def h(self, s_start, s_goal):
        heuristic_type = self.heuristic_type  # heuristic type

        if heuristic_type == "manhattan":
            return abs(s_goal[0] - s_start[0]) + abs(s_goal[1] - s_start[1])
        else:
            return math.hypot(s_goal[0] - s_start[0], s_goal[1] - s_start[1])

    def cost(self, s_start, s_goal):
        """
        Calculate Cost for this motion
        :param s_start: starting node
        :param s_goal: end node
        :return:  Cost for this motion
        :note: Cost function could be more complicate!
        """

        if self.is_collision(s_start, s_goal):
            return float("inf")

        return math.hypot(s_goal[0] - s_start[0], s_goal[1] - s_start[1])

    def is_collision(self, s_start, s_end):
        if s_start in self.obs or s_end in self.obs:
            return True

        if s_start[0] != s_end[0] and s_start[1] != s_end[1]:
            if s_end[0] - s_start[0] == s_start[1] - s_end[1]:
                s1 = (min(s_start[0], s_end[0]), min(s_start[1], s_end[1]))
                s2 = (max(s_start[0], s_end[0]), max(s_start[1], s_end[1]))
            else:
                s1 = (min(s_start[0], s_end[0]), max(s_start[1], s_end[1]))
                s2 = (max(s_start[0], s_end[0]), min(s_start[1], s_end[1]))

            if s1 in self.obs or s2 in self.obs:
                return True

        return False

    def get_neighbor(self, s):
        nei_list = set()
        for u in self.u_set:
            s_next = tuple([s[i] + u[i] for i in range(2)])
            if s_next not in self.obs:
                nei_list.add(s_next)

        return nei_list

    def extract_path(self):
        """
        Extract the path based on the PARENT set.
        :return: The planning path
        """

        path = [self.s_start]
        s = self.s_start

        for k in range(200):
            g_list = {}
            for x in self.get_neighbor(s):
                if not self.is_collision(s, x):
                    g_list[x] = self.g[x]
            s = min(g_list, key=g_list.get)
            path.append(s)
            if s == self.s_goal:
                break

        return list(path)

    def plot_path(self, path):
        px = [x[0] for x in path]
        py = [x[1] for x in path]
        plt.plot(px, py, linewidth=2)
        plt.plot(self.s_start[0], self.s_start[1], "bs")
        plt.plot(self.s_goal[0], self.s_goal[1], "gs")

    def plot_visited(self, visited):
        color = ['gainsboro', 'lightgray', 'silver', 'darkgray',
                 'bisque', 'navajowhite', 'moccasin', 'wheat',
                 'powderblue', 'skyblue', 'lightskyblue', 'cornflowerblue']

        if self.count >= len(color) - 1:
            self.count = 0

        for x in visited:
            plt.plot(x[0], x[1], marker='s', color=color[self.count])



In [91]:
%matplotlib
s_start = (round(dog.x,0), round(dog.y,0))
s_goal = (30,465)

dstar = DStar(s_start, s_goal, "euclidean")
dstar.run()

Using matplotlib backend: TkAgg


In [92]:
path = dstar.extract_path()

In [93]:
path

[(126, 341),
 (125, 342),
 (124, 343),
 (123, 344),
 (122, 345),
 (121, 346),
 (120, 347),
 (119, 348),
 (118, 349),
 (117, 350),
 (116, 351),
 (115, 352),
 (114, 353),
 (113, 354),
 (112, 355),
 (111, 356),
 (110, 357),
 (109, 358),
 (108, 359),
 (107, 360),
 (106, 361),
 (105, 362),
 (104, 363),
 (103, 364),
 (102, 365),
 (101, 366),
 (100, 367),
 (99, 368),
 (98, 369),
 (97, 370),
 (96, 371),
 (95, 372),
 (94, 373),
 (93, 374),
 (92, 375),
 (91, 376),
 (90, 377),
 (89, 378),
 (88, 379),
 (87, 380),
 (86, 381),
 (85, 382),
 (84, 383),
 (83, 384),
 (82, 385),
 (81, 386),
 (80, 387),
 (80, 388),
 (79, 389),
 (78, 390),
 (77, 391),
 (76, 392),
 (75, 393),
 (74, 394),
 (73, 395),
 (72, 396),
 (71, 397),
 (70, 398),
 (69, 399),
 (68, 400),
 (67, 401),
 (66, 402),
 (65, 403),
 (64, 404),
 (63, 405),
 (62, 406),
 (61, 407),
 (60, 408),
 (59, 409),
 (58, 410),
 (57, 411),
 (56, 412),
 (55, 413),
 (55, 414),
 (55, 415),
 (55, 416),
 (55, 417),
 (55, 418),
 (55, 419),
 (55, 420),
 (55, 421),
 

In [96]:
showmap(room, dog, path=path, goal=s_goal)

In [77]:
dog.x, dog.y

(11.944503092668516, 4.925711674215627)

In [95]:
MoveUsingPath(room, dog, path)

99.91761682570042, Current Y: 367.0264917508013
Goal X: 99, Goal Y: 368
Angle absolu -133.30713958537592
Angle relatif -359.63796281598616
Distance à parcourir 1.337811253528094
--------------
Current X: 98.93702287421387, Current Y: 367.98919110316115
Goal X: 98, Goal Y: 369
Angle absolu -132.8306097574734
Angle relatif -1.7636732884747062
Distance à parcourir 1.3783129153891702
--------------
Current X: 97.87941657337161, Current Y: 368.9864241448792
Goal X: 97, Goal Y: 370
Angle absolu -130.946138222654
Angle relatif -358.1163082211726
Distance à parcourir 1.3419051842825986
--------------
Current X: 97.0203277047279, Current Y: 370.08468511632765
Goal X: 96, Goal Y: 371
Angle absolu -138.10537637938606
Angle relatif -7.60771462579072
Distance à parcourir 1.370718775426763
--------------
Current X: 95.9030754252608, Current Y: 370.9879739724819
Goal X: 95, Goal Y: 372
Angle absolu -131.7439384465467
Angle relatif -353.6502596654786
Distance à parcourir 1.3563708578718683
-----------

True

In [89]:
len(path)

101

In [ ]:
# POURQUOI ????
# dog.human() # Ce que ça fait ? Nécessaire pour le plotrgb, mais fait apparaitre un humain qui se balade partout ?
dog.plotmap()
dog.plotlidar()
dog.plotRGB()
dog.plotdepth()
human.x, human.y = 10,18

In [ ]:
from scipy import signal

filter_arr = np.array([[0,1,0],[1,1,1],[0,1,0]])

counts = scipy.signal.convolve2d(dftest, filter_arr, mode='same')
counts

In [ ]:
dftest.to_csv("maps/mapnorm2.csv",sep=';', header=False, index=False)